In [1]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import operator
import random
import time

# Объявление всех функций и классов

Открываем файл

In [2]:
def open_plain_txt(path):
    file = open(path, 'r')
    text = file.read()
    file.close()
    return text

Получаем параметры, из str перегоняем в int

In [3]:
def get_params(lst):
    lst = list(filter(None, lst))
    for i in range(len(lst)):
        lst[i] = int(lst[i])
    return lst

Парсим херню, получаем список с параметрами задачи и список с объектами пицц

In [4]:
def parse_input(path):
    text = open_plain_txt(path)
    text = text.split('\n')
    params = text[0].split(' ')
    params = get_params(params)
    pizzas = text[1:]
    pizzas = list(filter(None,pizzas))
    objs = []
    for i in range(len(pizzas)):
        objs.append(Pizza(i, pizzas[i].split(' ')))
        
    return params, objs

Функция, чтобы выяснить качество и количество уникальных ингридиентров

In [5]:
def get_all_ing(pizza_list):
    all_ingredients = set()
    for i in range(len(pizza_list)):
        all_ingredients = all_ingredients | set(pizza_list[i].ingredients)
    return all_ingredients

Это для АНАЛиза, можете забить на эту функцию

In [6]:
def min_extra_groups(excess):
    mod = excess%4
    num_twos = 0
    num_threes = 0
    num_fours = 0
    groups_data = [0,0,0]
    if mod:
        if mod == 1:
            num_fours = int(excess/4)-1
            num_threes = 1
            num_twos = 1
        elif mod == 2:
            num_fours = int(excess/4)
            num_twos = 1
        elif mod == 3:
            num_fours = int(excess/4)
            num_threes = 1
    else:
        num_fours = int(excess/4)
    groups_data[0] = num_twos
    groups_data[1] = num_threes
    groups_data[2] = num_fours
    return groups_data

def max_extra_groups(excess):
    mod = excess%2
    num_twos = 0
    num_threes = 0
    groups_data = [0,0,0]
    if mod:
        num_twos = int(excess/2) - 1
        num_threes = 1
    else:
        num_twos = int(excess/2)
    groups_data[0] = num_twos
    groups_data[1] = num_threes
    return groups_data

Тоже для анализа

In [18]:
def exercise_info(params, pizza_list):
    pizza_amount = params[0]
    group_two = params[1]
    group_three = params[2]
    group_four = params[3]
    all_groups = group_two+group_three + group_four
    pizzas_needed = group_two*2+group_three*3+group_four*4
    excess = pizzas_needed - pizza_amount
    all_ingredients = get_all_ing(pizza_list)
    
    print(f'Доступно пицц: {pizza_amount}')
    print(f'Групп из 2-х человек: {group_two}')
    print(f'Групп из 3-х человек: {group_three}')
    print(f'Групп из 4-х человек: {group_four}\n')
    print(f'Количество пицц, нужных, чтобы накормить все группы: {pizzas_needed}\n')
    print(f'С помощью {pizza_amount} можно накормить:') 
    print(f'Максимум {all_groups - sum(min_extra_groups(excess))} из {all_groups} групп. ({min_extra_groups(excess)})')
    print(f'Минимум {all_groups - sum(max_extra_groups(excess))} из {all_groups} групп. ({max_extra_groups(excess)})\n')
    print(f'Тогда, без учета потерь на совпадающие ингридиенты:') 
    print(f'Максимальные очки = {100*(all_groups - sum(min_extra_groups(excess)))}')
    print(f'Минимальные очки = {100*(all_groups - sum(max_extra_groups(excess)))}')
    print(f'Всего уникальных ингридиентов {len(all_ingredients)}: {all_ingredients}')

Класс с пиццей, это мы обсуждали

In [19]:
class Pizza():
    def __init__(self, idx, lst):
        self.idx = idx
        self.num_ing = int(lst[0])
        self.ingredients = lst[1:]

Тут все более интересно, этот класс собирает группу, и потом в себя вбирает объекты пицц, более подробно при созвоне, если таковой случится

In [20]:
class ConstructGroup():
    
    def __init__(self, num_members):
        self.num_members = num_members
        
    def retrieval_method(self, pizza_list, ceil, eps):
        start = time.time()
        while True:
            pizzas_ordered = []
            init_set = set()
            all_ing = 0
            
            while len(pizzas_ordered) < self.num_members:
                random_selection = random.choice(pizza_list)
                if random_selection not in pizzas_ordered:
                    pizzas_ordered.append(random_selection)

            for i in range(self.num_members):
                init_set = init_set | set(pizzas_ordered[i].ingredients)
                all_ing += pizzas_ordered[i].num_ing
            if all_ing in ceil and (all_ing - len(init_set)) < eps:
                break
            if time.time() - start > 5:
                pizzas_ordered = None
                break
        self.score = len(init_set)
        return pizzas_ordered
    
    def get_pizza(self, pizza_list, ceil, eps):
        self.pizzas_eaten = self.retrieval_method(pizza_list, ceil, eps)
        
    def delete_pizza(self, pizza_list):
        for i in range(len(self.pizzas_eaten)):
            pizza_list.remove(self.pizzas_eaten[i])
    
#     def count_score(self):
#         self.score = 0
#         for i in range(len(self.pizzas_eaten)):
#             #переделать
            
#             self.score += self.pizzas_eaten[i].num_ing

Функция которая сортирует список пицц по кол-ву ингридиентов, напрямую не используется, но возможно когда нибудь пригодится

In [10]:
def sort_pizza_list(pizza_list):
    sorted_pizzas = sorted(pizza_list, key=operator.attrgetter('num_ing'))
    return sorted_pizzas

Туповатая функция, делает следующее: создаем группы (с одним кол-вом членов азазаз) с пиццами до тех пор, пока к жопе не припрет и вайл не сдохнет

In [11]:
def assemble_groups_cont(params, pizza_list, num_members, ceil, eps):
    groups = []
    scores = []
    score = 0
    for i in range(params[num_members-1]):
        obj = ConstructGroup(num_members)
        try:
            obj.get_pizza(pizza_list, ceil, eps)
        except:
            print('No more pizzas.')
            break
        if obj.pizzas_eaten and len(pizza_list)>0:
            groups.append(obj)
            try:
                obj.delete_pizza(pizza_list)
            except Exception as e:
                print(e)
            scores.append(obj.score)
            params[num_members-1] -= 1
#             print(i+1, len(pizza_list)) #Закоментировать чтобы циферками не пердело.
        else:
            print('Timeout')
            break
    return groups, scores

Создает группы с рандомным количеством членов. Эту функцию я не доделал, ну и хер с ней.

In [12]:
def assemble_groups_rand(params, pizza_list, ceil, eps):
    groups = []
    scores = []
    score = 0
    timeouts = 0
    iteration = 0
    while sum(params[1:4]) > 0:
        #Тут надо переделать, в прошлый раз получилось params = [12,14,-15]
        num_members = np.random.randint(2,5)
        obj = ConstructGroup(num_members)
        obj.get_pizza(pizza_list, ceil, eps)
        if obj.pizzas_eaten:
            timeouts = 0
            groups.append(obj)
            try:
                obj.delete_pizza(pizza_list)
            except Exception as e:
                print(e)
            scores.append(obj.score)
            params[num_members-1] -= 1
            iteration += 1
            print(iteration, len(pizza_list))
        else:
            timeouts += 1
            print(f'Timeout #{timeouts}')
        if timeouts > 10:
            print('Cycle ended')
            break
    return groups, scores

# Запускаем код

Сначала парсим говно

In [13]:
# params, pizza_list = parse_input('b_little_bit_of_everything.in')
params, pizza_list = parse_input('c_many_ingredients.in') 
sorted_pizzas = sort_pizza_list(pizza_list)
#params, pizza_list = parse_input('a_example')

Запускаем говно, чтобы заполнило все гуппы по 4. Можно было бы сделать функцию, но мне было лень. Оно там циферками напердит, первая - номер итерации, вторая - кол-во пицц оставшихся. Для дебага. Можно закоментировать в функции assemble_groups_cont

In [14]:
num_members = 4
ideal = 10
eps = 2
iteration = 0
groups_of_4 = []
scores4 = []
while params[num_members-1] > 0:
    ceil = np.arange(ideal-iteration, ideal+iteration+1)
    assembled = assemble_groups_cont(params, pizza_list, num_members, ceil, eps)
    groups_of_4 += assembled[0]
    scores4 += assembled[1]
    iteration += 1 

Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
No more pizzas.
No more pizzas.
No more pizzas.
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout
Timeout


Точно то же самое только для групп из трех.

In [ ]:
num_members = 3
ideal = 10
eps = 2
iteration = 0
groups_of_3 = []
scores3 = []
while params[num_members-1] > 0:
    ceil = np.arange(ideal-iteration, ideal+iteration+1)
    assembled = assemble_groups_cont(params, pizza_list, num_members, ceil, eps)
    groups_of_3 += assembled[0]
    scores3 += assembled[1]
    iteration += 1 

Для групп по двое. Дуэль. 4 это короче, голландский штурвал. Если виснет, значит ждите

In [ ]:
num_members = 2
ideal = 10
eps = 2
iteration = 0
groups_of_2 = []
scores2 = []
while params[num_members-1] > 0 and len(pizza_list) > 0:
    ceil = np.arange(ideal-iteration, ideal+iteration+1)
    assembled = assemble_groups_cont(params, pizza_list, num_members, ceil, eps)
    groups_of_2 += assembled[0]
    scores2 += assembled[1]
    iteration += 1 

#  Вывод результатов

In [ ]:
pizzas_delivered = len(groups_of_4)*4 + len(groups_of_3)*3 + len(groups_of_2)*2
overall_score = np.sum(np.asarray(scores4)**2) + np.sum(np.asarray(scores3)**2) + np.sum(np.asarray(scores2)**2)
groups = []
groups = groups_of_4 + groups_of_3 + groups_of_2
deliveries = len(groups)

print(f"Использовано пицц: {pizzas_delivered}")
print(f"Заполнено групп: {deliveries}")
print(f"Получено очков: {overall_score}")


Вывод результатов в файл

In [319]:
output_file = open("HashCode_out", "w+")
output_file.write(f"{deliveries}\n")

for i in range(len(groups)):
    string = f"{groups[i].num_members}"
    for j in range(len(groups[i].pizzas_eaten)):
        pizza = groups[i].pizzas_eaten[j].idx
        string += f" {pizza}"
    output_file.write(string + '\n')
output_file.close()

In [ ]:
#Гистограмма распределения количества ингридиентов.

nums = []
for i in range(params[0]):
    nums.append(pizza_list[i].num_ing)

plt.figure(1)
plt.clf()
plt.hist(nums, bins=np.arange(min(nums), max(nums)+2), align = 'left', edgecolor = 'black')
plt.grid(linestyle = '--')
plt.xlim(0,max(nums)+1)
plt.xlabel('Кол-во ингридиентов')
exercise_info(params, pizza_list)

In [254]:
# ideal = 10
# eps = 2
# iteration = 0
# groups = []
# scores = []
# while sum(params[1:4]) > 0:
#     ceil = np.arange(ideal-iteration, ideal+iteration+1)
#     assembled = assemble_groups_rand(params, pizza_list, ceil, eps)
#     groups += assembled[0]
#     scores += assembled[1]
#     iteration += 1 

In [231]:
# ceil = [8,9,10,11,12]
# eps = 3
# groups, scores = assemble_groups_rand(params, pizza_list, ceil, eps)